# Mask Recognition with tflite

This notebook leverages the public face mask detection dataset from Kaggle to train a model to identify masks in pictures.  The steps are as follows:

* 1) Imports and setup.
* 2) Fine tune the model. (MobileNetV2)
* 3) Optimize and export

## 1) Imports and Setup

In [ ]:
#basic imports 
import tensorflow as tf
assert tf.__version__.startswith('2')
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import files

In [ ]:
#mount my drive as this is where I have the dataset stored.
drive.mount('/content/drive')

In [ ]:
#verify TF verison, should be >= 2.0
tf.__version__

In [ ]:
# Set the base directory for the masks.
# Should contain 2 subfolders: [Masks, No masks]
base_dir = '/content/drive/My Drive/masks'

In [ ]:
#Use image size 224x224
IMAGE_SIZE = 224
BATCH_SIZE = 64

#Create data generator to normalize and split into training/test
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

#generate training data.
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

#generate test data
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

In [ ]:
#Check the shape.
#Should be ((Batch, Image_size, Image_size, #_Colors), (Batch, #_Classes))
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

In [ ]:
#write the labels to a .txt file for the app.
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

In [ ]:
#take a look
!cat labels.txt

## 2) Fine tune the Model


### 2a) Import and Setup Model

In [ ]:
#Set image shape
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

In [ ]:
#freeze the base of the model.
base_model.trainable = False

In [ ]:
#Add a classification head.
#Make sure the output layer size = your number of classes.  
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
#Compile model and set less.
#If you're using more than 2 classes use categorical_crossentropy for the loss instead.
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [ ]:
#take a look at the model. 
model.summary()

In [ ]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

### 2b) Train the Base Model


In [ ]:
#I used 5 epochs to prevent overfitting, on my dataset.
#You may have to change based on your data.
epochs = 5

#set history so I can plot the loss/accuracy
history = model.fit(train_generator, 
                    steps_per_epoch=len(train_generator), 
                    epochs=epochs, 
                    validation_data=val_generator, 
                    validation_steps=len(val_generator))

In [ ]:
#extract the accuracy and loss for training and validation sets.
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

#plot accuracy
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

#plot loss
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### 2c) Fine tune the model


In [ ]:
#unfreeze the top layers
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
#recompile the model, ideally with a lower learning rate.
model.compile(loss='binary_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

In [ ]:
#take a look.
model.summary()

In [ ]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

In [ ]:
#train the model some more.
#I used 5 more epochs here, but would've been fine with just 2.
history_fine = model.fit(train_generator, 
                         steps_per_epoch=len(train_generator), 
                         epochs=5, 
                         validation_data=val_generator, 
                         validation_steps=len(val_generator))

## 3) Optimize and Export

In [ ]:
#First I need to save the model.
saved_model_dir = 'save/fine_tuning'
tf.saved_model.save(model, saved_model_dir)

In [ ]:
#Now I can convert this to a TFlite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

#once its converted, write it to a file. 
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
#download the files so you can have them locally.
files.download('model.tflite')
files.download('labels.txt')

In [ ]:
#I also saved the model weights in h5 and json.
#These can be used with openCV if you want to run this on a local computer.
mask_json = model.to_json()
with open("mask_detection.json", "w") as json_file:
    json_file.write(mask_json)
model.save_weights("mask_detection.h5")